### MySQL 접속용 샘플 프로그램

##### 1. 접속 정보가 들어 있는 파일 읽기

In [1]:
import json
with open('./mysql.json') as f:
    config_str = f.read()
config = json.loads(config_str)
config

In [2]:
config

{'host': 'localhost',
 'user': 'ysuser',
 'password': 'yspass',
 'database': 'world',
 'port': 3306}

In [ ]:
#  이렇게 하니 로드 안됨
# a = json.load('mysql.json')
# a

##### 2. MySQL에 접속

In [5]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)
# conn = pymysql.connect(**config) 과 같이 사용할 수 있음

In [6]:
conn

##### 2. MySQL접속하기

In [7]:
# city table에서 10개 데이터 읽기
cur = conn.cursor()
sql = "SELECT * FROM city LIMIT 10;"
# ; 세미콜론은 프로그램안에서 생략가능
cur.execute(sql)
rows = cur.fetchall() # 조건에 맞는 tabel에 있는 데이터 몽땅다 
cur.close()

for row in rows:
    print(row)

(1, 'Kabul', 'AFG', 'Kabol', 1780000)
(2, 'Qandahar', 'AFG', 'Qandahar', 237500)
(3, 'Herat', 'AFG', 'Herat', 186800)
(4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800)
(5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)
(6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321)
(7, 'Haag', 'NLD', 'Zuid-Holland', 440900)
(8, 'Utrecht', 'NLD', 'Utrecht', 234323)
(9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843)
(10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)


In [9]:
import pandas as pd
df = pd.DataFrame(rows, columns=['id', 'name', 'country_code', 'district', 'population'])
df.set_index('id', inplace=True)
df

,name,country_code,district,population
id,,,,
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500
3,Herat,AFG,Herat,186800
4,Mazar-e-Sharif,AFG,Balkh,127800
5,Amsterdam,NLD,Noord-Holland,731200
6,Rotterdam,NLD,Zuid-Holland,593321
7,Haag,NLD,Zuid-Holland,440900
8,Utrecht,NLD,Utrecht,234323
9,Eindhoven,NLD,Noord-Brabant,201843


- 검색: 국가코드 KOR 의 인구수 TOP 10 

In [14]:
cur = conn.cursor()
# sql = "SELECT * FROM city WHERE countrycode='KOR' ORDER BY popilation DESC LIMIT 10;"
# 검색을 하면 sqllite나 자바는 ? 인데 파이썬은 %s 파라미터로 받겠다
sql = "SELECT * FROM city WHERE countrycode=%s ORDER BY population DESC LIMIT 10;"
cur.execute(sql, ('KOR',))  # 파라메터는 반드시 튜플로 전달해야 하기 때문에 ('KOR', ) 콤마를 적어준것임
rows = cur.fetchall()
cur.close()

In [15]:
for row in rows:
    print(row)

(2331, 'Seoul', 'KOR', 'Seoul', 9981619)
(2332, 'Pusan', 'KOR', 'Pusan', 3804522)
(2333, 'Inchon', 'KOR', 'Inchon', 2559424)
(2334, 'Taegu', 'KOR', 'Taegu', 2548568)
(2335, 'Taejon', 'KOR', 'Taejon', 1425835)
(2336, 'Kwangju', 'KOR', 'Kwangju', 1368341)
(2340, '수원', 'KOR', 'Kyonggi', 1200000)
(2337, 'Ulsan', 'KOR', 'Kyongsangnam', 1084891)
(2338, 'Songnam', 'KOR', 'Kyonggi', 869094)
(2339, 'Puchon', 'KOR', 'Kyonggi', 779412)


- 데이터 추가(Insert)

In [16]:
cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, '자바','KOR','Kyonggi', 6000000);"
# cur.execute(sql, ('KOR',))  # 파라메터는 반드시 튜플로 전달해야 하기 때문에 ('KOR', ) 콤마를 적어준것임
cur.execute(sql)  # 파라메터로 받을게 없기 때문에 sql만
# rows = cur.fetchall() # 결과로 오는게 없기 때문에 필요 없음
conn.commit() # DB가 변화되는 SQL(Insert, Update, Delete)에서는 필수
cur.close()

In [17]:
params = ('노드', 'KOR', 'Kyonggi', 6000000)

cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, %s, %s, %s, %s);"
cur.execute(sql, params) 
conn.commit()       # DB가 변화되는 SQL(Insert, Update, Delete)에서는 필수
cur.close()

In [19]:
params_list = [
    ('HTML', 'KOR', 'Kyonggi', 7000000), ('CSS', 'KOR', 'Kyonggi', 7200000)
]

In [20]:
for params in params_list:
    print(params)

('HTML', 'KOR', 'Kyonggi', 7000000)
('CSS', 'KOR', 'Kyonggi', 7200000)


In [21]:
params_list = [
    ('HTML', 'KOR', 'Kyonggi', 7000000), ('CSS', 'KOR', 'Kyonggi', 7200000)
]

cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, %s, %s, %s, %s);"

for params in params_list:
    cur.execute(sql, params) # 두줄이 들어감 html,css

conn.commit()
cur.close()

In [22]:
params_list = [
    ('HTML', 'KOR', 'Kyonggi', 7000000), ('CSS', 'KOR', 'Kyonggi', 7200000)
]

cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, %s, %s, %s, %s);"

# for params in params_list:
#     cur.execute(sql, params) # 두줄이 들어감

cur.executemany(sql, params_list) 
# for 문 대신 executemany 사용해도됨 
# 그러나 몇건은 가능하지만 대량은 어려울 수 있음. 코드가 안정적이지 않음. 100건,1000건일 경우는 사용하면 속도가 느림..
conn.commit()
cur.close()

- 데이터 수정(Update)

In [23]:
cur = conn.cursor()
sql = "UPDATE city SET `Name`='CSS3' WHERE id=5008;"
cur.execute(sql) 
conn.commit()
cur.close()

In [26]:
#  HTML -> HTML5 로 네임변경
params_list = [('HTHML5',5007), ('HTML5',5009)]
cur = conn.cursor()
# sql = "UPDATE city SET `Name`='HTML5' WHERE id=5007;"
sql = "UPDATE city SET `Name`=%s WHERE id=%s;"
# cur.execute(sql, params) 
cur.executemany(sql, params_list) 
conn.commit()
cur.close()

- 데이터 삭제(Delete)

In [27]:
cur = conn.cursor()
# sql = "DELETE FROM city WHERE id >= 5000;"
sql = "DELETE FROM city WHERE id >= %s;"
cur.execute(sql, (5000,)) 
conn.commit()
cur.close()

In [28]:
cur = conn.cursor()
sql = "SELECT * FROM city WHERE name=%s;"
cur.execute(sql, ('수원', ))  # 파라메터는 반드시 튜플로 전달
row = cur.fetchone()    # 하나의 데이터(레코드) / 한건만 오기 때문에 테이블에 수원은 한줄만있음
conn.commit()
cur.close()
row

(2340, '수원', 'KOR', 'Kyonggi', 1200000)

##### 4. MySQL 접속 해제

In [29]:
conn.close()